In [0]:
# Install required libraries
!pip install -q openai python-dotenv implicit scikit-learn scipy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import os
import json
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, ndcg_score
from implicit.als import AlternatingLeastSquares

import openai
import time
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import json
from decimal import Decimal

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Load environment variables
load_dotenv()

# Configure
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully")


✅ All libraries imported successfully


In [0]:
import os

# Load OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key is None:
    raise ValueError(
        "❌ OPENAI_API_KEY not found in environment variables.\n"
        "Please set it in your .env file or system environment.\n"
        "Create a .env file in the project root with: OPENAI_API_KEY=xxxxxxxx"
    )

print(f"✅ API key loaded from environment (key starts with: {openai_api_key[:20]}...)")

CONFIG = {
    'als': {
        'factors': 50,
        'regularization': 0.01,
        'iterations': 15,
        'alpha': 40.0
    },
    'llm': {
        'model': 'gpt-4o',
        'temperature': 0.3,
        'max_tokens': 2000
    },
    'recommendation': {
        'top_n': 5,
        'final_n': 3
    },
    'feature_engineering': {
        'recency_days': 90,
        'min_transactions': 3
    }
}

print("\n📊 Configuration:")
print(json.dumps(CONFIG, indent=2))



📊 Configuration:
{
  "als": {
    "factors": 50,
    "regularization": 0.01,
    "iterations": 15,
    "alpha": 40.0
  },
  "llm": {
    "model": "gpt-4o",
    "temperature": 0.3,
    "max_tokens": 2000
  },
  "recommendation": {
    "top_n": 5,
    "final_n": 3
  },
  "feature_engineering": {
    "recency_days": 90,
    "min_transactions": 3
  }
}


In [0]:
# ==============================================================================
# DATA LOADING - Databricks Unity Catalog Tables (SPARK VERSION)
# ==============================================================================
# Update these table names to match your Unity Catalog tables
# Format: catalog.schema.table_name

TRANSACTIONS_TABLE = 'workspace.default.transaction_data_1'
PRODUCTS_TABLE = 'workspace.default.zenith_bank_product_catalog'
CONVERSATIONS_TABLE = 'workspace.default.conversation_data_1'
CUSTOMERS_TABLE = 'workspace.default.customer_data_1'  # Optional

# ==============================================================================
# For CSV files in DBFS, use this format instead:
# TRANSACTIONS_PATH = '/dbfs/FileStore/tables/zenith_bank_transaction.csv'
# Then use: df_transactions = spark.read.csv(TRANSACTIONS_PATH, header=True, inferSchema=True)
# ==============================================================================

print("📂 Loading datasets from Unity Catalog...\n")

# Load data using Spark (NO pandas conversion for distributed processing)
try:
    # Load transactions
    print(f"Loading: {TRANSACTIONS_TABLE}")
    df_transactions = spark.table(TRANSACTIONS_TABLE)
    transactions_count = df_transactions.count()
    print(f"✅ Loaded {transactions_count:,} transactions")
    
    # Load products
    print(f"\nLoading: {PRODUCTS_TABLE}")
    df_products = spark.table(PRODUCTS_TABLE)
    products_count = df_products.count()
    print(f"✅ Loaded {products_count:,} products")
    
    # Load conversations
    print(f"\nLoading: {CONVERSATIONS_TABLE}")
    df_conversations = spark.table(CONVERSATIONS_TABLE)
    conversations_count = df_conversations.count()
    print(f"✅ Loaded {conversations_count:,} conversations")
    
    # Try loading customer demographics (optional)
    print(f"\nLoading: {CUSTOMERS_TABLE}")
    try:
        df_customers = spark.table(CUSTOMERS_TABLE)
        customers_count = df_customers.count()
        print(f"✅ Loaded {customers_count:,} customer demographics")
    except Exception as e:
        df_customers = None
        print(f"⚠️ Customer demographics table not found: {e}")
        print("   Proceeding without demographics data")
    
    # Display basic info using Spark operations
    print("\n" + "="*80)
    print("📊 DATASET OVERVIEW")
    print("="*80)
    print(f"Unique Customers: {df_transactions.select('Customer_ID').distinct().count():,}")
    print(f"Unique Products: {df_products.select('Product_ID').distinct().count():,}")
    
    # Get date range
    date_stats = df_transactions.agg(
        F.min('Date').alias('min_date'),
        F.max('Date').alias('max_date')
    ).collect()[0]
    print(f"Date Range: {date_stats['min_date']} to {date_stats['max_date']}")
    
    # Get total transaction value
    total_value = df_transactions.agg(F.sum('Trans_Amount').alias('total')).collect()[0]['total']
    print(f"Total Transaction Value: ₦{total_value:,.2f}")
    print("="*80)
    
except Exception as e:
    print(f"\n❌ Error loading data: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Verify table names exist in Unity Catalog")
    print("   2. Check you have READ permissions on these tables")
    print("   3. Run: spark.sql('SHOW TABLES IN workspace.default').show()")
    print("   4. Or run: spark.catalog.listTables('workspace.default')")

📂 Loading datasets from Unity Catalog...

Loading: workspace.default.transaction_data_1
✅ Loaded 125,549 transactions

Loading: workspace.default.zenith_bank_product_catalog
✅ Loaded 42 products

Loading: workspace.default.conversation_data_1
✅ Loaded 3,446 conversations

Loading: workspace.default.customer_data_1
✅ Loaded 1,000 customer demographics

📊 DATASET OVERVIEW
Unique Customers: 1,000
Unique Products: 42
Date Range: 2025-08-15 to 2025-11-13
Total Transaction Value: ₦19,040,566,404.07


In [0]:
# Quick data exploration
print("\n📊 TRANSACTIONS SAMPLE:")
display(df_transactions.head())

print("\n🏦 PRODUCTS SAMPLE:")
display(df_products.head())

print("\n👥 CUSTOMERS SAMPLE:")
display(df_customers.head())

print("\n💬 CONVERSATIONS SAMPLE:")
display(df_conversations.head())

print("\n📈 TRANSACTION STATISTICS:")
print(df_transactions.describe())

# Check for missing values
print("\n🔍 Missing Values:")
print(df_transactions.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df_transactions.columns]).show())


📊 TRANSACTIONS SAMPLE:


Row(Customer_ID='ZB000001', Trans_Amount=30126.97, Date=datetime.date(2025, 10, 1), Destination='Bookshop', Deb_or_credit='D', Narration='textbooks', Tran_Id='TR0000024', Category='Books', Account_Type='Aspire Account')


🏦 PRODUCTS SAMPLE:


Row(Product_ID='PROD001', Product_Name='Zenith Childrens Account (ZECA)', Product_Category='Savings Account', Age_Range='0-15 years', Opening_Balance='0', Minimum_Balance='0', Maximum_Balance='No limit', Currency='Naira, Dollar', Interest_Rate='Competitive', Monthly_Fee='0', Account_Maintenance_Fee='0', Key_Features='Zero opening balance, Scholarship opportunities, Education loan, Customized prepaid card, Annual Children Parade invitation', Digital_Channels='Zenith Mobile App, *966# EazyBanking, ZIVA', Card_Type='Customized Prepaid Card (upon parent request)', Target_Audience='Children aged 0-15 years', Special_Benefits='Teaches children to save, Parents can save for child future', Status='Active')


👥 CUSTOMERS SAMPLE:


Row(Customer_ID='ZB000001', Full_Name='Engr. Adewale', First_Name='Adewale', Last_Name='Yusuf', Gender='Male', Date_of_Birth=datetime.date(1994, 10, 14), Age=31, Phone_Number=80472958637, Email='adewale.yusuf@outlook.com', Address='24 Awolowo Ave', City='New Lagos', State='Kwara', Occupation='Civil Servant', Employment_Status='Employed', Income_Bracket='₦1,000,000+', Marital_Status='Single', Education_Level='BSc', Account_Type='Aspire Account', Account_Creation_Date=6080000000.0, BVN=93774863057, Status='Active')


💬 CONVERSATIONS SAMPLE:


Row(Conversation_ID='CONV00001', Customer_ID='ZB000001', Agent_ID='AGT90', Category='Complaints', Customer_Message='My debit card is not working', Customer_Sentiment='#######', Agent_Response='You can request via the mobile app', Agent_Time='#######', Channel='Phone Call', Status='Resolved', Satisfaction_Score=3)


📈 TRANSACTION STATISTICS:
DataFrame[summary: string, Customer_ID: string, Trans_Amount: string, Destination: string, Deb_or_credit: string, Narration: string, Tran_Id: string, Category: string, Account_Type: string]

🔍 Missing Values:
+-----------+------------+----+-----------+-------------+---------+-------+--------+------------+
|Customer_ID|Trans_Amount|Date|Destination|Deb_or_credit|Narration|Tran_Id|Category|Account_Type|
+-----------+------------+----+-----------+-------------+---------+-------+--------+------------+
|          0|           0|   0|          0|            0|        0|      0|       0|           0|
+-----------+------------+----+-----------+-------------+---------+-------+--------+------------+

None


In [0]:
def create_customer_product_interactions(df_custs, df_products, df_trans,
                                          openai_api_key=None,


🚀 Starting Hybrid interaction matrix creation...

🔨 Creating interaction matrix (Hybrid: OpenAI + Transaction Descriptions)...

   Products in catalog: 42

   → Identifying current products from Account_Type...
      Found current products for 571 customers

   → Preparing customer profiles...
      Total customers: 1,000
      Processing all 1,000 customers

   → Preparing product catalog and deriving rules from data...
   → Deriving business rules from product catalog...
      Generated 86 product-specific rules from catalog

   → Using OpenAI to score product fit (intelligent matching)...
      This may take time depending on sample size...
      Processing batch 1/34... ✓ (150 matches)
      Processing batch 2/34... ✓ (150 matches)
      Processing batch 3/34... ✓ (150 matches)
      Processing batch 4/34... ✓ (150 matches)
      Processing batch 5/34... ✓ (150 matches)
      Processing batch 6/34... ✓ (150 matches)
      Processing batch 7/34... ✓ (150 matches)
      Processing b

{'application/vnd.databricks.toolz-hint+json': {'trigger': 'hungCommand'}}

Python Execution Stuck!


 ✓ (150 matches)
      Processing batch 11/34... ✓ (150 matches)
      Processing batch 12/34... ✓ (150 matches)
      Processing batch 13/34... ✓ (150 matches)
      Processing batch 14/34... ✓ (150 matches)
      Processing batch 15/34... ✓ (150 matches)
      Processing batch 16/34... ✓ (150 matches)
      Processing batch 17/34... ✓ (150 matches)
      Processing batch 18/34... ✓ (150 matches)
      Processing batch 19/34... ✓ (150 matches)
      Processing batch 20/34... ✓ (150 matches)
      Processing batch 21/34... ✓ (150 matches)
      Processing batch 22/34... ✓ (150 matches)
      Processing batch 23/34... ✓ (150 matches)
      Processing batch 24/34... ✓ (150 matches)
      Processing batch 25/34... ✓ (150 matches)
      Processing batch 26/34... ✓ (150 matches)
      Processing batch 27/34... ✓ (150 matches)
      Processing batch 28/34... ✓ (150 matches)
      Processing batch 29/34... ✓ (150 matches)
      Processing batch 30/34... ✓ (150 matches)
      Processing batch 

{'application/vnd.databricks.toolz-hint+json': {'trigger': 'hungCommand'}}

Python Execution Stuck!


 ✓ (150 matches)
      Processing batch 33/34... ✓ (150 matches)
      Processing batch 34/34... ✓ (50 matches)

   → Computing transaction-based interaction scores...
      Analyzing transaction descriptions for product signals...
      Filtered to 125,549 transactions for sampled customers
      ⚠ Description field not found - using Account_Type only
      Matched 157,512 transactions to products
      Computing RFM (Recency, Frequency, Monetary) scores...
      Generated 1,231 transaction-based interactions

      Sample RFM scores:
+-----------+-----------------------------+-----------------+-----------------+--------------------+---------------+
|Customer_ID|Product_Name                 |trans_score      |transaction_count|total_amount        |days_since_last|
+-----------+-----------------------------+-----------------+-----------------+--------------------+---------------+
|ZB000094   |Aspire Account               |10.0             |151              |2.588046261E7       |0      

In [0]:
interaction_df=interaction_df.toPandas()
interaction_df.to_csv("interaction_df.csv", index=False)

In [0]:
def engineer_customer_features(df_trans, df_convs, df_custs=None, recency_days=90, customer_sample_size=1000):
    """
    Create comprehensive customer features for ML and LLM context.
    SPARK OPTIMIZED: Uses distributed processing
    """
    import time
    from pyspark.sql import Window
    from pyspark.sql.functions import col, count, sum as spark_sum, mean, stddev, expr
    from datetime import timedelta
    
    start_time = time.time()
    
    print(f"🔨 Engineering customer features for {customer_sample_size if customer_sample_size else 'ALL'} customers (Spark)...")
    
    # =========================================================================
    # SAMPLE CUSTOMERS FIRST (from df_custs - SOURCE TABLE)
    # =========================================================================
    if customer_sample_size and df_custs is not None:
        print(f"   → Sampling {customer_sample_size:,} customers from customer table...")
        total_customers = df_custs.count()
        sample_fraction = customer_sample_size / total_customers
        df_custs_sample = df_custs.sample(fraction=sample_fraction, seed=42).limit(customer_sample_size)
        sampled_customer_ids = [row.Customer_ID for row in df_custs_sample.select('Customer_ID').collect()]
        sampled_customer_ids_set = set(sampled_customer_ids)
        
        # Filter transactions and conversations to only include sampled customers
        df_trans = df_trans.filter(col('Customer_ID').isin(sampled_customer_ids))
        df_convs = df_convs.filter(col('Customer_ID').isin(sampled_customer_ids))
        df_custs = df_custs_sample
        
        print(f"      ✓ Sampled {len(sampled_customer_ids):,} customers")
        print(f"      ✓ Filtered to {df_trans.count():,} transactions")
        print(f"      ✓ Filtered to {df_convs.count():,} conversations")
    
    # =========================================================================
    # PREPARE TRANSACTION DATA
    # =========================================================================
    print("   → Preparing transaction data...")
    
    # Ensure Date is date type
    df_trans = df_trans.withColumn('Date', F.to_date(F.col('Date')))
    
    # Create debit/credit flags
    df_trans = df_trans.withColumn('is_debit', F.when(F.col('Deb_or_credit') == 'D', 1).otherwise(0))
    df_trans = df_trans.withColumn('is_credit', F.when(F.col('Deb_or_credit') == 'C', 1).otherwise(0))
    df_trans = df_trans.withColumn('debit_amount', F.col('Trans_Amount') * F.col('is_debit'))
    df_trans = df_trans.withColumn('credit_amount', F.col('Trans_Amount') * F.col('is_credit'))
    
    # Calculate recency
    max_date = df_trans.agg(F.max('Date')).collect()[0][0]
    recent_threshold = max_date - timedelta(days=recency_days)
    
    df_trans = df_trans.withColumn('is_recent', F.when(F.col('Date') >= F.lit(recent_threshold), 1).otherwise(0))
    df_trans = df_trans.withColumn('recent_debit', F.col('debit_amount') * F.col('is_recent'))
    
    # =========================================================================
    # 1. DATE RANGE FEATURES
    # =========================================================================
    print("   → Computing date range features...")
    
    date_ranges = df_trans.groupBy('Customer_ID').agg(
        F.min('Date').alias('min_date'),
        F.max('Date').alias('max_date')
    )
    
    date_ranges = date_ranges.withColumn(
        'date_span_days',
        F.when(F.datediff(F.col('max_date'), F.col('min_date')) == 0, 1)
         .otherwise(F.datediff(F.col('max_date'), F.col('min_date')))
    )
    
    # =========================================================================
    # 2. AGGREGATED FEATURES
    # =========================================================================
    print("   → Computing aggregated features...")
    
    agg_features = df_trans.groupBy('Customer_ID').agg(
        # Financial metrics
        F.sum('debit_amount').alias('total_debit'),
        F.sum('credit_amount').alias('total_credit'),
        F.mean('Trans_Amount').alias('avg_transaction'),
        F.stddev('Trans_Amount').alias('std_transaction'),
        F.expr('percentile_approx(Trans_Amount, 0.5)').alias('median_transaction'),
        F.count('Trans_Amount').alias('transaction_count'),
        
        # Debit/Credit counts
        F.sum('is_debit').alias('debit_count'),
        F.sum('is_credit').alias('credit_count'),
        
        # Diversity metrics
        F.countDistinct('Category').alias('unique_categories'),
        F.countDistinct('Destination').alias('unique_destinations'),
        
        # Recency
        F.sum('is_recent').alias('recent_transaction_count'),
        F.sum('recent_debit').alias('recent_debit'),
        
        # Account type (most common using first)
        F.first('Account_Type').alias('current_account')
    )
    
    # =========================================================================
    # 3. TOP CATEGORY FEATURES
    # =========================================================================
    print("   → Computing category preferences...")
    
    # Filter debit transactions
    debit_txns = df_trans.filter(F.col('is_debit') == 1)
    
    # Top category per customer (most frequent)
    window_spec = Window.partitionBy('Customer_ID', 'Category')
    category_counts = debit_txns.groupBy('Customer_ID', 'Category').agg(
        F.count('*').alias('category_count')
    )
    
    window_rank = Window.partitionBy('Customer_ID').orderBy(F.desc('category_count'))
    top_categories = category_counts.withColumn('rank', F.row_number().over(window_rank)) \
        .filter(F.col('rank') == 1) \
        .select('Customer_ID', F.col('Category').alias('top_category'))
    
    # Top category spending amount
    category_amounts = debit_txns.groupBy('Customer_ID', 'Category').agg(
        F.sum('Trans_Amount').alias('category_amount')
    )
    
    window_rank_amount = Window.partitionBy('Customer_ID').orderBy(F.desc('category_amount'))
    top_category_amounts = category_amounts.withColumn('rank', F.row_number().over(window_rank_amount)) \
        .filter(F.col('rank') == 1) \
        .select('Customer_ID', F.col('category_amount').alias('top_category_amount'))
    
    # Category concentration (simplified - top category count / total count)
    category_concentration = category_counts.join(
        debit_txns.groupBy('Customer_ID').agg(F.count('*').alias('total_count')),
        'Customer_ID'
    ).withColumn('category_concentration', F.col('category_count') / F.col('total_count'))
    
    window_rank_conc = Window.partitionBy('Customer_ID').orderBy(F.desc('category_concentration'))
    category_concentration = category_concentration.withColumn('rank', F.row_number().over(window_rank_conc)) \
        .filter(F.col('rank') == 1) \
        .select('Customer_ID', 'category_concentration')
    
    # =========================================================================
    # 4. MERGE BASE FEATURES
    # =========================================================================
    print("   → Merging all features...")
    
    df_features = agg_features.join(date_ranges.select('Customer_ID', 'date_span_days'), 'Customer_ID', 'left')
    df_features = df_features.join(top_categories, 'Customer_ID', 'left')
    df_features = df_features.join(top_category_amounts, 'Customer_ID', 'left')
    df_features = df_features.join(category_concentration, 'Customer_ID', 'left')
    
    # Fill nulls
    df_features = df_features.fillna({
        'std_transaction': 0,
        'top_category': 'Unknown',
        'top_category_amount': 0,
        'category_concentration': 0
    })
    
    # =========================================================================
    # 5. COMPUTED FEATURES
    # =========================================================================
    print("   → Computing derived features...")
    
    df_features = df_features.withColumn('net_balance', F.col('total_credit') - F.col('total_debit'))
    df_features = df_features.withColumn(
        'debit_credit_ratio',
        F.when(F.col('total_credit') > 0, F.col('total_debit') / F.col('total_credit')).otherwise(0)
    )
    df_features = df_features.withColumn(
        'transaction_frequency_days',
        F.col('transaction_count') / F.col('date_span_days')
    )
    df_features = df_features.withColumn('days_since_last_transaction', F.col('date_span_days'))
    
    # =========================================================================
    # 6. CONVERSATION FEATURES
    # =========================================================================
    print("   → Adding conversation features...")
    
    conv_count = df_convs.count()
    if conv_count > 0:
        conv_features = df_convs.groupBy('Customer_ID').agg(
            F.count('Category').alias('conversation_count'),
            F.first('Category').alias('top_inquiry_category'),
            F.count('Customer_Message').alias('message_count')
        )
        df_features = df_features.join(conv_features, 'Customer_ID', 'left')
        df_features = df_features.fillna({
            'conversation_count': 0,
            'message_count': 0,
            'top_inquiry_category': 'None'
        })
    else:
        df_features = df_features.withColumn('conversation_count', F.lit(0))
        df_features = df_features.withColumn('message_count', F.lit(0))
        df_features = df_features.withColumn('top_inquiry_category', F.lit('None'))
    
    # =========================================================================
    # 7. DEMOGRAPHIC FEATURES (from df_custs - SOURCE TABLE)
    # =========================================================================
    print("   → Adding demographic features...")
    
    if df_custs is not None:
        demo_cols = ['Customer_ID']
        available_cols = df_custs.columns
        
        if 'Age' in available_cols: demo_cols.append('Age')
        if 'Occupation' in available_cols: demo_cols.append('Occupation')
        if 'Income_Bracket' in available_cols: demo_cols.append('Income_Bracket')
        if 'State' in available_cols: demo_cols.append('State')
        if 'Gender' in available_cols: demo_cols.append('Gender')
        if 'Location' in available_cols: demo_cols.append('Location')
        
        demo_features = df_custs.select(demo_cols)
        df_features = df_features.join(demo_features, 'Customer_ID', 'left')
    
    # =========================================================================
    # 8. FINAL DERIVED FEATURES
    # =========================================================================
    print("   → Computing final derived features...")
    
    df_features = df_features.withColumn(
        'financial_velocity',
        F.col('transaction_count') / (F.col('days_since_last_transaction') + 1)
    )
    df_features = df_features.withColumn(
        'spending_consistency',
        F.col('std_transaction') / (F.col('avg_transaction') + 1)
    )
    df_features = df_features.withColumn(
        'engagement_score',
        (F.col('transaction_count') * 0.4) + 
        (F.col('unique_categories') * 10 * 0.3) + 
        (F.col('conversation_count') * 5 * 0.3)
    )
    
    elapsed_time = time.time() - start_time
    feature_count = df_features.count()
    col_count = len(df_features.columns)
    
    print(f"✅ Engineered {feature_count:,} customer profiles with {col_count} features")
    print(f"   ⚡ Completed in {elapsed_time:.2f} seconds ({feature_count/elapsed_time:.0f} customers/sec)")
    
    return df_features


# ============================================================================
# EXECUTE
# ============================================================================

print("🚀 Starting customer feature engineering...\n")

customer_features = engineer_customer_features(
    df_transactions,      # Transactions table (filtered by sampled customers)
    df_conversations,     # Conversations table (filtered by sampled customers)
    df_customers,         # CUSTOMERS TABLE - SOURCE TABLE for sampling
    recency_days=CONFIG['feature_engineering']['recency_days'],
    customer_sample_size=1000  # Sample 1000 customers
)

print("\n📊 Customer Features Sample:")
customer_features.show(10, truncate=False)

print("\n📈 Feature Statistics:")
customer_features.describe().show()

print("\n📋 Feature Columns:")
print(f"Total columns: {len(customer_features.columns)}")
for col in customer_features.columns:
    print(f"  - {col}")

🚀 Starting customer feature engineering...

🔨 Engineering customer features for 1000 customers (Spark)...
   → Sampling 1,000 customers from customer table...
      ✓ Sampled 1,000 customers
      ✓ Filtered to 125,549 transactions
      ✓ Filtered to 3,446 conversations
   → Preparing transaction data...
   → Computing date range features...
   → Computing aggregated features...
   → Computing category preferences...
   → Merging all features...
   → Computing derived features...
   → Adding conversation features...
   → Adding demographic features...
   → Computing final derived features...
✅ Engineered 1,000 customer profiles with 33 features
   ⚡ Completed in 1.95 seconds (513 customers/sec)

📊 Customer Features Sample:
+-----------+--------------------+--------------------+------------------+------------------+------------------+-----------------+-----------+------------+-----------------+-------------------+------------------------+--------------------+--------------------------+

In [0]:
customer_features1=customer_features.toPandas()
customer_features1.to_csv("customer_features.csv", index=False)


In [0]:
# ============================================================================  
# CONFIGURATION  
# ============================================================================  

# Load API key from environment (already loaded in cell 4)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if OPENAI_API_KEY is None:
    raise ValueError("❌ OPENAI_API_KEY not set. Please set the environment variable first.")

OPENAI_MODEL = "gpt-4o"


📋 Step 1: Preparing data...
✅ Cleaned: 4,801 interactions

📋 Step 2: Creating indexes...
  ✓ Customers: 1,000
  ✓ Products: 30

🔨 Step 3: Training ALS...
✅ Trained. RMSE: 0.2467

📋 Step 4: Generating recommendations...
  ✓ Users: 999, Items: 29
  ✓ Computed 4,995 recommendations

💾 Saving ALS table...
✅ Table 1 saved: als_recommendations_table

🤖 Step 5: Generating LLM explanations (limiting to 20 customers)...
  → Selecting top 20 customers...
  ✓ Processing 20 customers, 60 recommendations...
  → Fetching customer features...
  ✓ Loaded 20 customer profiles

  🔄 Starting LLM generation at 13:08:23...
  → Progress: 1/60 (1%) | Elapsed: 0.0s | Est. remaining: 0.0s
    ✓ Generated explanation for ZB000071 - Timeless Savings Account (Rank 3)
    ✓ Generated explanation for ZB000071 - Aspire Account (Rank 2)
    ✓ Generated explanation for ZB000071 - Timeless Current Account (Rank 1)
    ✓ Generated explanation for ZB000101 - Timeless Savings Account (Rank 3)
  → Progress: 5/60 (8%) | El

In [0]:
interaction_df.show()

+-----------+--------------------+------------------+
|Customer_ID|        Product_Name| interaction_score|
+-----------+--------------------+------------------+
|   ZB000001|       Personal Loan|2.1000000000000005|
|   ZB000001|Zenith Individual...|               2.7|
|   ZB000001|Zenith Internet B...|2.4000000000000004|
|   ZB000001|Zenith Mobile Ban...|               2.7|
|   ZB000001|Zenith Salary Sav...|2.4000000000000004|
|   ZB000002|SME Grow My Biz A...|2.4000000000000004|
|   ZB000002|            SME Loan|2.1000000000000005|
|   ZB000002|Zenith Individual...|2.1000000000000005|
|   ZB000002|Zenith Internet B...|2.4000000000000004|
|   ZB000002|Zenith Mobile Ban...|               2.7|
|   ZB000003|       Personal Loan|1.8000000000000003|
|   ZB000003|Zenith Individual...|2.4000000000000004|
|   ZB000003|Zenith Internet B...|2.4000000000000004|
|   ZB000003|Zenith Mobile Ban...|               2.7|
|   ZB000003| Zenith Virtual Card|2.1000000000000005|
|   ZB000004|       Personal